In [1]:
import time
start = time.time()
from processing import train_data, embed
end = time.time()
print(end - start)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231903 entries, 0 to 260086
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   embedded_state   231903 non-null  object
 1   embedded_action  231903 non-null  object
 2   reward           231903 non-null  object
dtypes: object(3)
memory usage: 310.1 MB
None
16.62066960334778


In [2]:
import numpy as np
from random import randint
import pandas as pd
from functools import partial

## Environment

In [15]:
class itemEnv():
    def __init__(self, alpha = 0.4, sigma = 0.9): # albha parameter in consine similarity
        self.observation_space = train_data
        self.item_info = embed
        self.current_state = None
        self.init_state = self.reset()
        self.sigma = sigma
        self.alpha = alpha
        self.embedding_dim = feature_num
        self.reward_val = 1
#         self.rewards, self.avg_state, self.avg_action, self.nx_size = self.avg_group()
    
    def reset(self):
        init_state = self.observation_space['embedded_state'].sample(1).values[0]
#         init_state = self.observation_space.loc[7,'embedded_state']
        self.current_state = init_state
        return init_state

#     def feedback(self, pair):  # pair: state-action pair
#         prob = list()
#         denominator = 0.
#         max_prob = 0.
#         result = 0.
#         feed_back = ""
#         for r, s, a in zip(self.rewards, self.avg_state, self.avg_action):
#             numerator = np.dot(pair[0], s.T) / np.linalg.norm(pair[0], 2) + np.dot(pair[1], a.T) / np.linalg.norm(pair[1], 2)
#             denominator += numerator
#             prob.append(numerator)
#             if numerator > max_prob:
#                 max_prob = numerator
#                 feed_back = r
#         prob = prob/denominator
# #         for p, r in zip(prob, self.rewards):
# #             for k in range(1):
# #                 result += p * self.reward_val * np.power(self.sigma, k) * int(r.split(',')[k])
        
#         result = self.reward_val * int(feed_back.split(',')[0])
#         print(prob)
#         return feed_back, result

    @staticmethod
    def cos_sim(group, pair):
        max_prob = 0.
        # nx_size = len(group[1])
        s_i = np.array(group[1]['embedded_state'].values.tolist()) # 2 dim array:0-->sample, 1-->feature_num (N,(7*12))
        s_t = pair[0]                                              # 1 dim array:0-->sample, (7*12,)
        a_i = np.array(group[1]['embedded_action'].values.tolist()) #(N,9,7)
        a_t = pair[1]                                               #(7,)
        norm_si = np.linalg.norm(s_i, 2, axis = 1)                 # 1 dim (N,)
        norm_st = np.linalg.norm(s_t, 2, axis = 0)                 # 1 dim (1,)
        norm_ai = np.linalg.norm(a_i, 2, axis = 2)                 # 2 dim (N, 9)
        norm_at = np.linalg.norm(a_t, 2, axis = 0)                 # 1 dim (1,)
        
        #first term: (N,1), second term: (N,9)
        cos = env.alpha * (np.dot(s_i, s_t)/(norm_si * norm_st + 1e-10))[:,np.newaxis] + (1 - env.alpha) * np.dot(a_i, a_t)/(norm_ai * norm_at + 1e-10)  #1 dim
        # cos = cos / np.sum(cos)
        #cos: (N,9)
        max_id = np.argmax(cos)
        id_x = int(max_id / cos.shape[1])
        id_y = max_id % cos.shape[1]
        # print('i am here!')
        return (np.max(cos), id_x, id_y)
    
    def feedback_2(self, pair):
        data = train_data.groupby(['reward'])
#         denominator = 0.
        reward_ind = -1
        max_prob = 0.
        result = 0.
        feed_back = ""
        fnc = partial(self.cos_sim, pair = pair)
        prob = map(fnc, data)
        for tup, group in zip(prob, data):
            if tup[0] > max_prob:
                max_prob = tup[0]
                reward_ind = tup[2]  #id_y
                feed_back = group[0]          #index: reward str
#                 feed_back = group[1].iloc[tup[1], 2]
#                 idx = tup[1]

        r = 1 * int(feed_back.split(',')[reward_ind])  #reward_val
        result =  r if r > 0 else -1

        return feed_back, result
            

    def step(self, action):   #action: 1 dim array
        feed_back, result = self.feedback_2((self.current_state, action))
#         for i,r in enumerate([feed_back.split(',')[0]]): #0---> one action
        if result == 1:     #reward_val
            tmp = np.append(self.current_state, action)
            tmp = tmp[self.embedding_dim:]
            self.current_state = tmp  # state: 1 dim array
#         else:
#             self.current_state = self.observation_space['embedded_state'].sample(1).values[0]

        return result, self.current_state, feed_back
        
#     def avg_group(self):
#         nx_size = list()
#         avg_state = list()
#         avg_action = list()
#         rewards = list()
#         for reward, group in self.observation_space.groupby(['reward']):
#             nx_size.append(group.shape[0])
# #             state = np.mean(data['embedded_action'], axis = 0)
#             norm_s = np.linalg.norm(np.array(group['embedded_state'].values.tolist()), 2, axis = 1)
#             norm_s = np.where(norm_s == 0, 0.001, norm_s)
#             state = np.sum(group['embedded_state'] / norm_s) / group.shape[0]
#             norm_a = np.linalg.norm(np.array(group['embedded_action'].values.tolist()), 2, axis = 1)
#             norm_a = np.where(norm_a == 0, 0.001, norm_a)
#             action = np.sum(group['embedded_action'] / norm_a) / group.shape[0]
#             avg_state.append(state)
#             avg_action.append(action)
#             rewards.append(reward)
            
#         return rewards, avg_state, avg_action, nx_size
def gen_action(item_id):
    return embed[item_id]

In [16]:
env = itemEnv()

In [9]:
env.reset()
start = time.time()
# action = train_data['embedded_action'].sample(1).values[0][5]
# action = train_data.loc[0,'embedded_action'][0]
action = gen_action(str(randint(1, 381)))
r,s,f = env.step(action)
end = time.time()
r, s, f,end - start

(-1, array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.07092139,  2.46203633, -0.72387844, -0.12012139,
         1.63520295, -0.73754373, -2.24228081,  0.97162306,  0.92553209,
        -0.72387844, -0.10724431,  1.96710076, 

In [7]:
action = gen_action(str(randint(1, 381)))
r,s,f= env.step(action)
r, s, f

(1, array([ 0.07092139,  1.69378421, -0.72387844, -0.10893131,  1.20847718,
         0.15651593,  0.72406985,  0.07092139,  0.54140602, -0.72387844,
        -0.11866091,  1.73732535, -0.82606993, -2.24228081,  0.07092139,
        -0.61097216, -0.72387844, -0.12115068,  1.52213885, -1.04436125,
        -2.24228081,  1.87232473, -0.61097216, -0.72387844, -0.12357091,
         1.2230661 , -0.93494067, -2.24228081,  0.07092139, -0.61097216,
        -0.72387844, -0.12258235,  1.53308054, -1.02236716, -0.75910548,
         1.87232473,  0.15727996, -0.72387844, -0.11585022,  1.29601067,
        -0.53959694, -0.75910548,  0.97162306,  0.92553209, -0.72387844,
        -0.12226144,  1.55861114, -0.69410541, -0.75910548,  1.87232473,
        -0.61097216, -0.72387844, -0.11926497,  0.36961457, -0.77438383,
         0.72406985,  1.87232473,  0.92553209, -0.72387844, -0.11692224,
         0.96776008, -0.8805053 ,  0.72406985,  1.87232473, -0.61097216,
        -0.72387844, -0.11296006,  1.54402223, -

In [20]:
train_data.groupby('reward').size()

reward
0,0,0,0,0,0,0,0,0    21658
0,0,1,0,0,0,0,0,0     5885
0,1,0,0,0,0,0,0,0     6027
0,1,0,1,0,0,1,0,0        1
0,1,1,0,0,0,0,0,0     2940
1,0,0,0,0,0,0,0,0     7280
1,0,1,0,0,0,0,0,0     3228
1,1,0,0,0,0,0,0,0     4519
1,1,0,0,1,0,0,0,1        1
1,1,1,0,0,0,0,0,0     5187
1,1,1,0,0,1,0,0,0     4040
1,1,1,0,1,0,0,0,0     3852
1,1,1,0,1,1,0,0,0     3051
1,1,1,1,0,0,0,0,0     4639
1,1,1,1,0,1,0,0,0     3212
1,1,1,1,1,0,0,0,0     4639
1,1,1,1,1,1,0,0,0     3385
1,1,1,1,1,1,0,0,1     4730
1,1,1,1,1,1,0,1,0     7954
1,1,1,1,1,1,0,1,1     6974
1,1,1,1,1,1,1,0,0    10164
1,1,1,1,1,1,1,0,1     8643
1,1,1,1,1,1,1,1,0    33067
1,1,1,1,1,1,1,1,1    26984
dtype: int64

## Network

In [5]:
import torch
from torch import nn
from torch.nn import functional as act
from torch.distributions import Categorical
from torch.autograd import Variable

In [6]:
def try_gpu(): #single gpu
    i = 0
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

try_gpu()

device(type='cuda', index=0)

In [25]:
class LSTM(nn.Module):
    def __init__(self, inpt_sz, hidden_sz, hidden_l1, hidden_l2, oupt_sz): #embedding dim, , , 381
        super().__init__()
        #input_gate
        device = try_gpu()
        init_weights = partial(self.init_, device = device)
#         self.init_state = None
        self.reward = []
        self.inpt_sz = inpt_sz
        self.hidden_sz = hidden_sz
        self.hidden_l1 = hidden_l1
        self.hidden_l2 = hidden_l2
        self.oupt_sz = oupt_sz
        self.W_xi, self.W_hi, self.bias_i = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #forget_gate
        self.W_xf, self.W_hf, self.bias_f = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #output_gate
        self.W_xo, self.W_ho, self.bias_o = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #candidate memory cell
        self.W_xc, self.W_hc, self.bias_c = (init_weights((inpt_sz, hidden_sz)), init_weights(
                                            (hidden_sz, hidden_sz)), init_weights((1, hidden_sz))
                                            )
        #output_layer
        self.W_o1, self.W_o2, self.bias_o1, self.bias_o2 = (init_weights((hidden_sz, hidden_l1)), init_weights(
                                            (hidden_l1, hidden_l2)), init_weights((1, hidden_l1)), 
                                            init_weights((1, hidden_l2))
                                            ) 
        self.W_o3, self.bias_o3 = (init_weights((hidden_l2, oupt_sz)), 
                                            init_weights((1, oupt_sz)))
    @staticmethod
    def init_(shape, device):
#         param = torch.tensor(shape)
        def xvaier(param):
            return nn.init.xavier_uniform_(param)
#         param = xvaier(torch.rand(shape, device = device))
        param = torch.randn(shape, device = device) * 0.01
        return nn.Parameter(param)
    
    def forward(self, X, init_state = None):  #X: batch size, seq size, input size
        batch_size, seq_size, _ = X.shape
        hidden_sz = self.hidden_sz
#         oupts = []
        if init_state == None:         #H, C actually are constant, not trainable
            H, C = (torch.zeros(batch_size, hidden_sz, device = X.device),
                    torch.zeros(batch_size, hidden_sz, device = X.device)
                   )
        else:
            H, C = init_state    # in some circumstance
#         softmax = act.softmax(dim = 1)
        for seq in range(seq_size):
            x_t = X[:,seq,:]
            I = torch.sigmoid(x_t @ self.W_xi + H @ self.W_hi + self.bias_i)
            F = torch.sigmoid(x_t @ self.W_xf + H @ self.W_hf + self.bias_f)
            O = torch.sigmoid(x_t @ self.W_xo + H @ self.W_ho + self.bias_o)
            C_tilda = torch.tanh(x_t @ self.W_xc + H @ self.W_hc + self.bias_c)
            
            C = F * C + I * C_tilda
            H = torch.tanh(C) + O
            hidden_layer1 = torch.sigmoid(H @ self.W_o1 + self.bias_o1)
            hidden_layer2 = torch.relu(hidden_layer1 @ self.W_o2 + self.bias_o2)
            output = act.softmax(hidden_layer2 @ self.W_o3 + self.bias_o3, dim = 1)
#             output = torch.sigmoid(hidden_layer2 @ self.W_o3 + self.bias_o3)
#             output = torch.relu(hidden_layer1 @ self.W_o2 + self.bias_o2)
#             oupts.append(output)
            
#         self.init_state = (H, C)
#         return torch.cat(oupts, dim = 0)
        return output, (H, C)
        
        
    
    

In [26]:
feature_num = 30

In [27]:
actor = LSTM(feature_num, 64, 128, 256, 382)
# actor.load_state_dict(torch.load('modelpara.pth'))

In [28]:
def sgd(params, lr, batch_size):
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [29]:
#train
start = time.time()
device = try_gpu()
# loss_fn = nn.NLLLoss(reduction='sum')
state_num = 20
max_episodes = 50
max_episodes_len = 5
gamma = 0.9
optimizer = torch.optim.SGD(actor.parameters(), lr = 0.01)
for episode in range(max_episodes):
    env.reset()
    state = env.current_state
    for j in range(2):
    #     env.current_state = train_data.loc[7,'embedded_state']
        env.current_state = state
#         G_t = 0.
    #     bat_x = []
    #     target = []
        policy_loss_l = []
        reward_l = []
        optimizer.zero_grad()
        for i in range(max_episodes_len):
            n_state = np.reshape(env.current_state, [-1, state_num, feature_num])
#             n_state = torch.from_numpy(n_state).cuda().to(torch.float32)
            n_state = torch.from_numpy(n_state).to(device).to(torch.float32)
#             bat_x.append(n_state)
            prob, _= actor(n_state)

#             prob = act.softmax(prob, dim = 1)
            m = Categorical(prob)
#             action = m.sample() if np.random.random() > 0.2 else torch.randint(0, 381, [1]).cuda()  #0~380 [low, high)
            action = m.sample() if np.random.random() > 0.2 else torch.randint(0, 382, [1]).to(device)  #0~380 [low, high)
    #         action = m.sample()
    #         target.append(action)
            with torch.no_grad():
#                 embed_action = gen_action(str(action.item() + 1))  #action : char
                embed_action = gen_action(str(action.item()))  #action : char
                r, _, _ = env.step(embed_action)
#             G_t = gamma*G_t + r
            reward_l.append(r)
            policy_loss_l.append(-m.log_prob(action))
            print(f'{action.item()}:{r}',end = ' ')
#         print(f'\nproba:{prob[0][action.item()]}')
        
        for k in range(max_episodes_len - 1):
            reward_l[max_episodes_len-k-2] = reward_l[max_episodes_len-k-2] + gamma * reward_l[max_episodes_len-k-1]
        policy_loss_l = [a * b for a, b in zip(policy_loss_l, reward_l)]

        loss = torch.cat(policy_loss_l).sum()  
        loss.backward()
    #     bat_x = torch.cat(bat_x, 0)
    #     target = torch.cat(target, -1)
    #     pred, _ = actor(bat_x)
    #     log_prob = torch.log(pred)
    #     loss = loss_fn(G_t * log_prob, target)

    #     loss.backward()
    #     sgd(actor.parameters(), 0.1, max_episodes_len)
        optimizer.step()

#         pr, _= actor(n_state)
#         with torch.no_grad():
#             pr = act.softmax(pr, dim = 1)
#         print(f'after:{pr[0][action.item()]}')
        print(f'total reward:{sum(reward_l)}, loss:{loss.data}')

end = time.time()
end - start
# aa = torch.cat(aa, 0)

339:-1 186:-1 36:1 224:1 205:-1 total reward:-0.8460999999999997, loss:-5.061864376068115
92:1 227:-1 202:-1 219:-1 79:1 total reward:-2.9539000000000004, loss:-17.56693458557129
24:1 203:-1 36:1 177:-1 147:1 total reward:2.4661, loss:14.642876625061035
246:-1 56:1 308:1 238:1 170:-1 total reward:2.9539000000000004, loss:17.615970611572266
220:-1 231:-1 13:-1 273:-1 144:-1 total reward:-13.1441, loss:-78.23262023925781
59:-1 130:-1 47:-1 325:-1 204:-1 total reward:-13.1441, loss:-78.15579223632812
144:-1 151:-1 359:-1 168:-1 63:-1 total reward:-13.1441, loss:-77.99214172363281
222:-1 153:-1 205:-1 297:1 279:-1 total reward:-6.266100000000001, loss:-37.27794647216797
182:-1 24:-1 222:-1 6:-1 139:1 total reward:-4.9539, loss:-29.53072166442871
166:-1 352:-1 40:1 354:-1 149:-1 total reward:-7.7241, loss:-45.93189239501953
270:-1 373:-1 24:1 97:-1 374:-1 total reward:-7.7241, loss:-45.97041702270508
61:-1 43:-1 56:-1 40:-1 9:1 total reward:-4.9539, loss:-29.356725692749023
311:-1 195:-1 20

569.6392147541046

In [30]:
n_state = np.reshape(train_data.loc[10,'embedded_state'], [-1, state_num, feature_num])
n_state = torch.from_numpy(n_state).cuda().to(torch.float32)
prob, _= actor(n_state)
prob.sort()[1]

tensor([[182, 166, 222, 259, 288, 256, 124, 371, 220,  42,  59,  93, 279, 180,
         205, 359, 274, 348, 292,  46, 229, 164, 339, 202, 330, 263, 272, 153,
         231, 236, 347, 337,  17, 376, 298, 364, 285,  77,  57,  47, 144, 283,
         187, 345, 130, 344, 270, 133, 131, 346, 241, 223, 191, 305, 117,  14,
         273, 320, 299, 257,  11, 178, 149, 212,  48, 331, 217, 173, 333, 278,
         363, 159, 171,  13, 225, 227, 342, 271, 374, 119, 327,  43, 282, 190,
         341, 141, 353, 318,  55, 167, 261, 176, 194, 322, 151,   5, 295, 201,
         281, 326, 138, 314, 352, 186, 158,  97, 198, 302, 122, 310, 199, 356,
         163, 291, 316, 268, 293, 373, 313,   6, 154, 349,   3, 248, 160, 235,
         311,  54, 336, 362, 325, 192, 262,  67, 324,  60,  25, 204, 379,  35,
         290,  24, 104, 360, 372, 246, 183, 267, 177,  38, 238,  44, 269, 206,
          40, 224, 103, 354, 109, 312,  83, 116, 251, 143, 328, 100, 155, 214,
         179, 157,  45, 213, 211,  22, 355, 209,  32

In [32]:
n_state = np.reshape(train_data.loc[99,'embedded_state'], [-1, state_num, feature_num])
n_state = torch.from_numpy(n_state).cuda().to(torch.float32)
prob, _= actor(n_state)
prob.sort()[1]

tensor([[182, 166, 222, 259, 288, 256, 124, 371, 220,  42,  59,  93, 279, 180,
         205, 359, 274, 348, 292,  46, 229, 164, 339, 202, 330, 263, 272, 153,
         231, 236, 347, 337,  17, 376, 298, 364, 285,  77,  57,  47, 144, 283,
         187, 345, 130, 344, 270, 133, 131, 346, 241, 223, 191, 305, 117,  14,
         273, 320, 299, 257,  11, 178, 149, 212,  48, 331, 217, 173, 333, 278,
         363, 159, 171,  13, 225, 227, 342, 271, 374, 119, 327,  43, 282, 190,
         341, 141, 353, 318,  55, 167, 261, 176, 194, 322, 151,   5, 295, 201,
         281, 326, 138, 314, 352, 186, 158,  97, 198, 302, 122, 310, 199, 356,
         163, 291, 316, 268, 293, 373, 313,   6, 154, 349,   3, 248, 160, 235,
         311,  54, 336, 362, 325, 192, 262,  67, 324,  60,  25, 204, 379,  35,
         290,  24, 104, 360, 372, 246, 183, 267, 177,  38, 238,  44, 269, 206,
          40, 224, 103, 354, 109, 312,  83, 116, 251, 143, 328, 100, 155, 214,
         179, 157,  45, 213, 211,  22, 355, 209,  32

In [19]:
train_data.loc[10,'embedded_state'], train_data.loc[10,'state']

(array([ 0.97162306, -0.61097216, -0.72387844, -0.12398621,  1.2741273 ,
        -0.44447251, -0.75910548, -0.82978028, -0.61097216, -0.72387844,
        -0.12344275,  0.59574275, -0.36199468, -0.75910548,  0.07092139,
         2.07791027, -0.72387844, -0.12306521,  1.26318561,  0.36875888,
        -0.75910548,  0.07092139,  1.69378421, -0.72387844, -0.10893131,
         1.20847718,  0.15651593,  0.72406985,  1.87232473,  0.15727996,
        -0.72387844, -0.1105289 ,  2.03639811, -0.49011024, -2.24228081,
         0.97162306, -0.61097216, -0.72387844, -0.12460418,  1.43825259,
        -0.80352599, -2.24228081,  0.97162306, -0.61097216, -0.72387844,
        -0.12244425,  1.31424681, -0.67211132, -2.24228081,  1.87232473,
        -0.2268461 , -0.72387844, -0.11831516,  1.46013597, -0.96188342,
        -0.75910548,  0.97162306, -0.2268461 , -0.72387844, -0.12726186,
         1.0078796 , -0.3652938 , -0.75910548,  0.07092139,  2.8461624 ,
        -0.72387844, -0.12359178,  0.93858225, -1.0

In [13]:
train_data.loc[1,'embedded_state'], train_data.loc[1,'state']

(array([1.0000e+00, 0.0000e+00, 1.0000e+00, 4.1750e-03, 4.3600e-01,
        1.4400e+03, 2.0000e+00, 2.0000e+00, 7.0000e+00, 1.0000e+00,
        3.3960e-03, 2.0100e-01, 1.6621e+04, 3.0000e+00, 3.0000e+00,
        0.0000e+00, 1.0000e+00, 7.7120e-03, 7.2100e-01, 9.5400e+02,
        1.0000e+00, 4.0000e+00, 0.0000e+00, 1.0000e+00, 4.9990e-03,
        6.4500e-01, 5.9600e+02, 1.0000e+00, 4.0000e+00, 4.0000e+00,
        1.0000e+00, 2.5387e-02, 8.5100e-01, 2.9500e+02, 1.0000e+00,
        4.0000e+00, 7.0000e+00, 1.0000e+00, 6.4600e-03, 5.3200e-01,
        1.5570e+03, 2.0000e+00, 2.0000e+00, 7.0000e+00, 1.0000e+00,
        6.5770e-03, 6.7400e-01, 2.9540e+03, 1.0000e+00, 2.0000e+00,
        7.0000e+00, 1.0000e+00, 6.5770e-03, 6.7400e-01, 2.9540e+03,
        1.0000e+00, 2.0000e+00, 2.0000e+00, 1.0000e+00, 1.5764e-02,
        8.3300e-01, 2.9540e+03, 1.0000e+00, 2.0000e+00, 2.0000e+00,
        1.0000e+00, 1.5764e-02, 8.3300e-01, 2.9540e+03, 1.0000e+00,
        4.0000e+00, 0.0000e+00, 1.0000e+00, 3.55

In [33]:
prob

tensor([[0.0027, 0.0028, 0.0026, 0.0026, 0.0027, 0.0025, 0.0026, 0.0029, 0.0027,
         0.0027, 0.0027, 0.0025, 0.0028, 0.0025, 0.0025, 0.0026, 0.0027, 0.0025,
         0.0026, 0.0028, 0.0028, 0.0026, 0.0026, 0.0027, 0.0026, 0.0026, 0.0027,
         0.0029, 0.0027, 0.0027, 0.0027, 0.0027, 0.0026, 0.0026, 0.0027, 0.0026,
         0.0028, 0.0028, 0.0026, 0.0028, 0.0026, 0.0027, 0.0024, 0.0025, 0.0026,
         0.0026, 0.0024, 0.0025, 0.0025, 0.0028, 0.0027, 0.0026, 0.0027, 0.0027,
         0.0026, 0.0025, 0.0028, 0.0025, 0.0026, 0.0024, 0.0026, 0.0027, 0.0027,
         0.0027, 0.0027, 0.0027, 0.0027, 0.0026, 0.0030, 0.0027, 0.0026, 0.0028,
         0.0026, 0.0027, 0.0027, 0.0027, 0.0027, 0.0025, 0.0027, 0.0027, 0.0028,
         0.0027, 0.0027, 0.0026, 0.0027, 0.0027, 0.0026, 0.0027, 0.0027, 0.0027,
         0.0028, 0.0027, 0.0027, 0.0024, 0.0027, 0.0027, 0.0027, 0.0026, 0.0027,
         0.0027, 0.0026, 0.0026, 0.0026, 0.0026, 0.0026, 0.0027, 0.0027, 0.0027,
         0.0027, 0.0026, 0.0

In [46]:
train_data.loc[7,'action']

[1, 11, 25, 57, 51, 72, 149, 152, 233]

In [179]:
gen_action('239')

array([ 1.87232473,  2.07791027, -0.72387844, -0.12089336,  0.78539864,
       -0.0766214 ,  0.72406985])

In [178]:
gen_action('134')

array([ 1.87232473, -0.61097216, -0.72387844, -0.12432599,  0.73069021,
       -0.85796136, -0.75910548])

In [47]:
train_data.loc[7,'reward']

'1,1,0,0,0,0,0,0,0'

In [33]:
env.current_state = train_data.loc[0,'embedded_state']
action = gen_action('151')
r,s,f= env.step(action)
r, s, f

(-1, array([3.33333333e-01, 3.33333333e-01, 0.00000000e+00, 8.35436814e-04,
        6.36897767e-01, 1.75459899e-02, 5.00000000e-01, 3.33333333e-01,
        6.66666667e-01, 0.00000000e+00, 1.95080676e-03, 7.09753231e-01,
        1.35571611e-01, 1.00000000e+00, 3.33333333e-01, 6.66666667e-01,
        0.00000000e+00, 1.95080676e-03, 7.09753231e-01, 1.35571611e-01,
        1.00000000e+00, 3.33333333e-01, 6.66666667e-01, 0.00000000e+00,
        1.95080676e-03, 7.09753231e-01, 1.35571611e-01, 1.00000000e+00,
        1.00000000e+00, 1.11111111e-01, 0.00000000e+00, 9.23104185e-04,
        5.19388954e-01, 3.61240969e-02, 1.00000000e+00, 1.00000000e+00,
        1.11111111e-01, 0.00000000e+00, 9.23104185e-04, 5.19388954e-01,
        3.61240969e-02, 1.00000000e+00, 3.33333333e-01, 2.22222222e-01,
        0.00000000e+00, 1.63742707e-03, 9.78848414e-01, 1.70238601e-01,
        0.00000000e+00, 3.33333333e-01, 0.00000000e+00, 0.00000000e+00,
        1.30244215e-03, 9.11868390e-01, 1.76067027e-03, 0.00

In [22]:
torch.save(actor.state_dict(),'modelpara.pth')
# actor = actor.load_state_dict(torch.load('modelpara.pth'))

In [50]:
train_data.loc[7,'embedded_state']

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [13]:
G_t * pred

tensor([[-161.3674, -155.0946, -156.8075,  ..., -153.8318, -187.5206,
         -160.8493],
        [-157.4997, -150.8772, -152.9312,  ..., -150.8477, -182.5345,
         -156.8122],
        [-156.8636, -150.4019, -152.0855,  ..., -150.4251, -181.2172,
         -156.2092],
        ...,
        [-155.1541, -148.4594, -150.3322,  ..., -148.8944, -179.0215,
         -154.5119],
        [-155.1541, -148.4594, -150.3322,  ..., -148.8944, -179.0215,
         -154.5119],
        [-155.1541, -148.4594, -150.3322,  ..., -148.8944, -179.0215,
         -154.5119]], device='cuda:0', grad_fn=<MulBackward0>)

In [13]:
target

tensor([358, 307], device='cuda:0')

In [45]:
actor.parameters()

<generator object Module.parameters at 0x0000023CC7FCE480>

In [ ]:
def compare(group, x):
    max_prob = 0.
    nx_size = len(group)
    
    for i in range(nx_size):
        tmp = group.loc[i,'two'] + x
        if tmp > max_prob:
            max_prob = tmp
    print(max_prob)
    return max_prob
        
    
pool = ProcessPoolExecutor(max_workers=8)
for _ in pool.map(compare, (aa.groupby('one'), 1)):
    pass

In [55]:
for group in aa.groupby('one'):   
    print(group[1])
aa.groupby('one').size()

  one              two  three
0   1  [1.0, 1.0, 1.0]      3
2   1  [0.0, 0.0, 0.0]      5
  one              two  three
1   2  [0.0, 0.0, 0.0]      4
3   2  [1.0, 1.0, 1.0]      8


one
1    2
2    2
dtype: int64

In [67]:
for group in aa.groupby('one'):   
    i = np.argmax(group[1]['three'].values.tolist())
    print(i)
    print(group[1].iloc[i,1])

1
[1.5 1.5 1.5]
0
[0. 0. 0.]


In [43]:
for group in aa.groupby('one'):
    print(np.dot(np.array(group[1]['two'].values.tolist()), np.ones(3)))


[3. 0.]
[0. 3.]


In [44]:
for group in aa.groupby('one'):
    print(np.array(group[1]['two'].values.tolist()))

[[1. 1. 1.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 1. 1.]]


In [48]:
for group in aa.groupby('one'):
    a = np.array(group[1]['two'].values.tolist())
    n = np.linalg.norm(a,2,axis = 1)
    d = np.dot(a, np.ones(3))
    print(d/n)

[1.73205081        nan]
[       nan 1.73205081]


D:\Applications\Anaconda\anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [14]:
from multiprocessing import Pool
import long_time_task
import os

In [42]:
dd = np.array([[1,2,3],[4,5,6]])
np.max(dd)

6

In [ ]:
norm_2 = np.linalg.norm(np.array(train_data['embedded_action'].values.tolist()), 2, axis = 1)
norm_2 = np.where(norm_2 == 0, 0.001, norm_2)
norm_2.shape

In [ ]:
np.linalg.norm(train_data.loc[0,'embedded_state'],2)

In [ ]:
np.sum(train_data['embedded_action'] / norm_2)

In [ ]:
bb = train_data['embedded_action'] / norm_2

In [ ]:
np.dot(bb[5], bb[5].T)

In [ ]:
bb.shape

In [ ]:
np.array([[],2,3])

In [ ]:
print(train_data.info(memory_usage = 'deep'))

In [ ]:
# train_data['reward'] = train_data['reward'].apply(lambda row: str(row))
group_data = train_data.groupby('reward')  
for index, data in group_data:
    print(type(index))
#     print(j)

In [ ]:
group_data.size()